In [23]:
# Import TopologicPy modules. This is not needed on other computers
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")


def ExportTopologicToGLB(
    topologies,
    filepath,
    # color keys (RGB or RGBA or hex)
    vertexColorKey: str = "vColor",
    edgeColorKey: str = "eColor",
    faceColorKey: str = "fColor",
    topologyColorKey: str = "color",
    # opacity keys (separate from color)
    vertexOpacityKey: str = "vOpacity",
    edgeOpacityKey: str = "eOpacity",
    faceOpacityKey: str = "fOpacity",
    topologyOpacityKey: str = "opacity",
    # defaults
    default_vertex_color=(220, 80, 80, 255),
    default_edge_color=(80, 220, 80, 255),
    default_face_color=(80, 80, 220, 255),
    include_vertices: bool = True,
    include_edges: bool = True,
    include_faces: bool = True,
    triangulate_faces: bool = True,
    double_sided: bool = True,
    silent: bool = False,
):
    """
    Consolidated GLB exporter for TopologicPy.

    It examines each input topology and queries only what is appropriate:
      - Cluster/CellComplex: Cells, Faces, Edges, Vertices
      - Cell: Faces, Edges, Vertices
      - Face: Edges, Vertices
      - Edge: Vertices
      - Vertex: nothing further

    The glTF contains:
      - Node hierarchy that mirrors the discovered structure.
      - POINTS (vertices), LINES (edges), TRIANGLES (faces) for rendering.
      - Colors and separate opacity per entity (keys above).
      - Dictionaries embedded in `extras` on nodes and primitives.
      - For each face primitive, `extras["polygonRings"] = {"outer":[...], "holes":[[...]...]}`.
      - A sidecar JSON of the full hierarchy and dictionaries, embedded as a bufferView and referenced at `asset.extras["topologicMetadata"]`.

    Requires:
        pip install pygltflib numpy
    """
    import os
    import json
    import numpy as np
    from pygltflib import (
        GLTF2, Scene, Node, Mesh, Buffer, BufferView, Accessor, Asset, Primitive, Material,
        ARRAY_BUFFER, ELEMENT_ARRAY_BUFFER, FLOAT, UNSIGNED_SHORT, UNSIGNED_INT, VEC3, VEC4
    )

    MODE_POINTS = 0
    MODE_LINES = 1
    MODE_TRIANGLES = 4

    # TopologicPy
    from topologicpy.Topology import Topology
    from topologicpy.Vertex import Vertex as TVertex
    from topologicpy.Edge import Edge as TEdge
    from topologicpy.Face import Face as TFace
    from topologicpy.Wire import Wire as TWire
    from topologicpy.Cell import Cell as TCell
    from topologicpy.CellComplex import CellComplex as TCellComplex
    from topologicpy.Cluster import Cluster as TCluster
    from topologicpy.Dictionary import Dictionary

    # -------------------- helpers: colors, opacity, dicts, coords --------------------
    def _color_to_rgba(value, default):
        if value is None:
            return np.array(default, dtype=np.uint8)
        if isinstance(value, (list, tuple)):
            if len(value) == 3:
                return np.array([value[0], value[1], value[2], 255], dtype=np.uint8)
            if len(value) == 4:
                return np.array(value, dtype=np.uint8)
        if isinstance(value, str):
            s = value.strip()
            if s.startswith("#"):
                s = s[1:]
                if len(s) == 6:
                    r = int(s[0:2], 16); g = int(s[2:4], 16); b = int(s[4:6], 16)
                    return np.array([r, g, b, 255], dtype=np.uint8)
                if len(s) == 8:
                    r = int(s[0:2], 16); g = int(s[2:4], 16); b = int(s[4:6], 16); a = int(s[6:8], 16)
                    return np.array([r, g, b, a], dtype=np.uint8)
        return np.array(default, dtype=np.uint8)

    def _parse_opacity(val, default_alpha255):
        if val is None:
            return float(default_alpha255) / 255.0
        if isinstance(val, str):
            s = val.strip()
            if s.endswith("%"):
                try:
                    return max(0.0, min(1.0, float(s[:-1]) / 100.0))
                except Exception:
                    return float(default_alpha255) / 255.0
            try:
                x = float(s)
                if 0.0 <= x <= 1.0: return x
                if 1.0 < x <= 100.0: return x / 100.0
                if 100.0 < x <= 255.0: return x / 255.0
            except Exception:
                return float(default_alpha255) / 255.0
        if isinstance(val, (int, np.integer)):
            a = int(val)
            if a in (0, 1) and val in (0, 1): return float(a)
            return max(0, min(255, a)) / 255.0
        if isinstance(val, float):
            if 0.0 <= val <= 1.0: return float(val)
            if 1.0 < val <= 100.0: return val / 100.0
            if 100.0 < val <= 255.0: return val / 255.0
        return float(default_alpha255) / 255.0

    def _dict_value(obj, key):
        try:
            d = Topology.Dictionary(obj)
            if d:
                return Dictionary.ValueAtKey(d, key)
        except Exception:
            pass
        return None

    def _topologic_dict_to_python(d):
        if d is None:
            return {}
        try:
            keys = Dictionary.Keys(d)
            vals = Dictionary.Values(d)
            out = {}
            for k, v in zip(keys, vals):
                out[str(k)] = _jsonify(v)
            return out
        except Exception:
            try:
                js = Dictionary.ToJSON(d)
                return json.loads(js)
            except Exception:
                return {}

    def _entity_dictionary(obj):
        try:
            return _topologic_dict_to_python(Topology.Dictionary(obj))
        except Exception:
            return {}

    def _jsonify(x):
        if isinstance(x, (str, int, float, bool)) or x is None:
            return x
        if isinstance(x, (list, tuple)):
            return [_jsonify(i) for i in x]
        if isinstance(x, dict):
            return {str(k): _jsonify(v) for k, v in x.items()}
        try:
            return x.item()
        except Exception:
            return str(x)

    def _resolve_rgba_with_separate_opacity(obj, color_key, opacity_key, default_rgba, topo_fallback=None):
        col_val = _dict_value(obj, color_key)
        rgba = _color_to_rgba(col_val, default_rgba)
        base_a255 = int(rgba[3])
        op_val = _dict_value(obj, opacity_key)
        if op_val is not None:
            a = _parse_opacity(op_val, base_a255)
            rgba[3] = int(round(255.0 * max(0.0, min(1.0, a))))
            return rgba
        if topo_fallback is not None:
            topo_op = _dict_value(topo_fallback, topologyOpacityKey)
            if topo_op is not None:
                a = _parse_opacity(topo_op, base_a255)
                rgba[3] = int(round(255.0 * max(0.0, min(1.0, a))))
                return rgba
        return rgba

    def _coords_of_vertex(v):
        return [float(TVertex.X(v)), -float(TVertex.Z(v)), float(TVertex.Y(v))]

    # rings from a Face
    def _face_rings(face):
        rings = {"outer": [], "holes": []}
        try:
            outer = TFace.ExternalBoundary(face)
            if outer:
                ovs = TWire.Vertices(outer)
                rings["outer"] = [_coords_of_vertex(v) for v in ovs]
        except Exception:
            pass
        try:
            holes = TFace.InternalBoundaries(face)
            if holes:
                for hw in holes:
                    hvs = TWire.Vertices(hw)
                    rings["holes"].append([_coords_of_vertex(v) for v in hvs])
        except Exception:
            pass
        return rings

    # -------------------- helpers: buffers/accessors/materials --------------------
    binary_blob = bytearray()
    bufferViews = []
    accessors = []
    material_cache = {}

    def _append_buffer(data_bytes):
        start = len(binary_blob)
        binary_blob.extend(data_bytes)
        return start

    def _add_accessor_for_array(array, target=None, type_str=VEC3, componentType=FLOAT):
        byte_data = array.tobytes(order="C")
        start = _append_buffer(byte_data)
        bufferViews.append(BufferView(buffer=0, byteOffset=start, byteLength=len(byte_data), target=target))
        bv_index = len(bufferViews) - 1
        count = array.shape[0]
        kwargs = dict(bufferView=bv_index, byteOffset=0, componentType=componentType, count=count, type=type_str)
        if type_str == VEC3 and componentType == FLOAT and count > 0:
            kwargs["min"] = array.min(axis=0).tolist()
            kwargs["max"] = array.max(axis=0).tolist()
        acc = Accessor(**kwargs)
        accessors.append(acc)
        return len(accessors) - 1

    def _material_for(blend: bool, base_alpha: float):
        key = (bool(blend), round(float(base_alpha), 3), bool(double_sided))
        if key in material_cache:
            return material_cache[key]
        mat = Material(
            pbrMetallicRoughness={
                "baseColorFactor": [1.0, 1.0, 1.0, max(0.0, min(1.0, float(base_alpha)))],
                "metallicFactor": 0.0,
                "roughnessFactor": 1.0,
            },
            alphaMode="BLEND" if blend else "OPAQUE",
            doubleSided=bool(double_sided),
        )
        gltf.materials.append(mat)
        idx = len(gltf.materials) - 1
        material_cache[key] = idx
        return idx

    # -------------------- hierarchy bookkeeping and IDs --------------------
    # Stable integer IDs per kind
    next_ids = {"cellcomplex": 0, "cluster": 0, "cell": 0, "face": 0, "edge": 0, "vertex": 0}

    def _new_id(kind):
        i = next_ids[kind]
        next_ids[kind] += 1
        return i

    # sidecar metadata structure
    meta = {
        "cellcomplexes": [],
        "clusters": [],
        "cells": [],
        "faces": [],
        "edges": [],
        "vertices": [],
    }

    # -------------------- glTF containers --------------------
    gltf = GLTF2(asset=Asset(version="2.0"))
    gltf.materials = []
    meshes = []
    nodes = []

    # allow single topology
    if not isinstance(topologies, (list, tuple)):
        topologies = [topologies]

    # -------------------- export routines per level --------------------
    total_points = 0
    total_lines = 0
    total_tris = 0

    def export_vertices(parent_topo, vs, parent_topo_for_fallback):
        nonlocal total_points
        if not include_vertices or not vs:
            return None
        positions = np.array([_coords_of_vertex(v) for v in vs], dtype=np.float32)
        rgba_u8 = np.array(
            [_resolve_rgba_with_separate_opacity(v, vertexColorKey, vertexOpacityKey, default_vertex_color, parent_topo_for_fallback)
             for v in vs],
            dtype=np.uint8
        )
        colors = (rgba_u8.astype(np.float32) / 255.0)
        pos_acc = _add_accessor_for_array(positions, ARRAY_BUFFER, VEC3, FLOAT)
        col_acc = _add_accessor_for_array(colors, ARRAY_BUFFER, VEC4, FLOAT)
        any_transparent = np.any(colors[:, 3] < 0.9999)
        mat_idx = _material_for(any_transparent, base_alpha=1.0)
        prim = Primitive(
            attributes={"POSITION": pos_acc, "COLOR_0": col_acc},
            mode=MODE_POINTS,
            material=mat_idx,
        )
        prim.extras = {"vertexDictionaries": [_entity_dictionary(v) for v in vs]}
        mesh = Mesh(primitives=[prim], name="vertices")
        meshes.append(mesh)
        mesh_idx = len(meshes) - 1
        node = Node(name="Vertices", mesh=mesh_idx)
        nodes.append(node)
        node_idx = len(nodes) - 1
        total_points += positions.shape[0]

        # record vertices in sidecar
        for v in vs:
            vid = _new_id("vertex")
            meta["vertices"].append({
                "id": vid,
                "dict": _entity_dictionary(v),
            })
        return node_idx

    def export_edges(parent_topo, es, parent_topo_for_fallback):
        nonlocal total_lines
        if not include_edges or not es:
            return None
        line_positions = []
        line_colors = []
        line_indices = []
        edge_dicts = []
        idx = 0
        any_transparent = False
        for e in es:
            try:
                sv, ev = TEdge.StartVertex(e), TEdge.EndVertex(e)
            except:
                continue
            rgba_u8 = _resolve_rgba_with_separate_opacity(e, edgeColorKey, edgeOpacityKey, default_edge_color, parent_topo_for_fallback)
            col = (rgba_u8.astype(np.float32) / 255.0).tolist()
            if rgba_u8[3] < 255:
                any_transparent = True
            line_positions.extend([_coords_of_vertex(sv), _coords_of_vertex(ev)])
            line_colors.extend([col, col])
            line_indices.extend([idx, idx + 1])
            edge_dicts.append(_entity_dictionary(e))
            # sidecar metadata
            eid = _new_id("edge")
            meta["edges"].append({
                "id": eid,
                "dict": edge_dicts[-1],
            })
            idx += 2

        if not line_positions:
            return None

        line_positions = np.array(line_positions, dtype=np.float32)
        line_colors = np.array(line_colors, dtype=np.float32)
        if idx <= 65535:
            idx_array = np.array(line_indices, dtype=np.uint16); idx_ct = UNSIGNED_SHORT
        else:
            idx_array = np.array(line_indices, dtype=np.uint32); idx_ct = UNSIGNED_INT
        pos_acc = _add_accessor_for_array(line_positions, ARRAY_BUFFER, VEC3, FLOAT)
        col_acc = _add_accessor_for_array(line_colors, ARRAY_BUFFER, VEC4, FLOAT)
        start = _append_buffer(idx_array.tobytes(order="C"))
        bufferViews.append(BufferView(buffer=0, byteOffset=start, byteLength=idx_array.nbytes, target=ELEMENT_ARRAY_BUFFER))
        accessors.append(Accessor(bufferView=len(bufferViews)-1, byteOffset=0, componentType=idx_ct, count=idx_array.shape[0], type="SCALAR"))
        idx_acc = len(accessors) - 1
        mat_idx = _material_for(any_transparent, base_alpha=1.0)

        prim = Primitive(
            attributes={"POSITION": pos_acc, "COLOR_0": col_acc},
            indices=idx_acc,
            mode=MODE_LINES,
            material=mat_idx,
        )
        prim.extras = {"edgeDictionaries": edge_dicts}
        mesh = Mesh(primitives=[prim], name="edges")
        meshes.append(mesh)
        mesh_idx = len(meshes) - 1
        node = Node(name="Edges", mesh=mesh_idx)
        nodes.append(node)
        node_idx = len(nodes) - 1
        total_lines += idx_array.shape[0] // 2
        return node_idx

    def export_face(face, parent_topo_for_fallback):
        nonlocal total_tris
        # triangulated for rendering
        try:
            V, F = Topology.MeshData(face, triangulate=triangulate_faces)
        except TypeError:
            data = Topology.MeshData(face)
            if isinstance(data, dict):
                V, F = data.get("vertices", []), data.get("faces", [])
            else:
                V, F = data
        V = [[vert[0],-vert[2], vert[1]] for vert in V] # Transpose Axes
        V = np.asarray(V, dtype=np.float32)
        F = np.asarray(F)
        if V.size == 0 or F.size == 0:
            return None
        if len(F.shape) == 2 and F.shape[1] != 3:
            tris = []
            for face_idx_list in F:
                for k in range(1, len(face_idx_list) - 1):
                    tris.append([face_idx_list[0], face_idx_list[k], face_idx_list[k + 1]])
            F = np.asarray(tris, dtype=np.int64)
        idx_ct = UNSIGNED_SHORT if V.shape[0] <= 65535 else UNSIGNED_INT
        idx_array = F.flatten().astype(np.uint16 if idx_ct == UNSIGNED_SHORT else np.uint32)

        rgba_u8 = _resolve_rgba_with_separate_opacity(face, faceColorKey, faceOpacityKey, default_face_color, parent_topo_for_fallback)
        cols = np.tile((rgba_u8.astype(np.float32) / 255.0), (V.shape[0], 1)).astype(np.float32)

        pos_acc = _add_accessor_for_array(V, ARRAY_BUFFER, VEC3, FLOAT)
        col_acc = _add_accessor_for_array(cols, ARRAY_BUFFER, VEC4, FLOAT)

        start = _append_buffer(idx_array.tobytes(order="C"))
        bufferViews.append(BufferView(buffer=0, byteOffset=start, byteLength=idx_array.nbytes, target=ELEMENT_ARRAY_BUFFER))
        accessors.append(Accessor(bufferView=len(bufferViews)-1, byteOffset=0, componentType=idx_ct, count=idx_array.shape[0], type="SCALAR"))
        idx_acc = len(accessors) - 1

        any_transparent = rgba_u8[3] < 255
        mat_idx = _material_for(any_transparent, base_alpha=1.0)

        rings = _face_rings(face)
        prim = Primitive(
            attributes={"POSITION": pos_acc, "COLOR_0": col_acc},
            indices=idx_acc,
            mode=MODE_TRIANGLES,
            material=mat_idx,
        )
        prim.extras = {
            "faceDictionary": _entity_dictionary(face),
            "polygonRings": rings
        }
        mesh = Mesh(primitives=[prim], name="face")
        meshes.append(mesh)
        mesh_idx = len(meshes) - 1
        node = Node(name="Face", mesh=mesh_idx)
        nodes.append(node)
        node_idx = len(nodes) - 1
        total_tris += F.shape[0]

        # sidecar metadata
        fid = _new_id("face")
        meta["faces"].append({
            "id": fid,
            "dict": _entity_dictionary(face),
            "rings": rings
        })
        return node_idx

    def export_faces(parent_topo, fs, parent_topo_for_fallback):
        if not include_faces or not fs:
            return None
        child_ids = []
        for f in fs:
            n = export_face(f, parent_topo_for_fallback)
            if n is not None:
                child_ids.append(n)
        if not child_ids:
            return None
        node = Node(name="Faces", children=child_ids)
        nodes.append(node)
        return len(nodes) - 1

    # Recursive traversal depending on what the object actually contains
    def export_topology(topo, name_hint="topology"):
        # Detect what child collections are available
        try:
            cells = Topology.Cells(topo)
        except Exception:
            cells = []
        try:
            faces = Topology.Faces(topo)
        except Exception:
            faces = []
        try:
            edges = Topology.Edges(topo)
        except Exception:
            edges = []
        try:
            verts = Topology.Vertices(topo)
        except Exception:
            verts = []

        # Node extras for this topology level
        topo_dict = _entity_dictionary(topo)

        # Sidecar: classify and register
        # Try to classify by availability of children
        kind = None
        if cells:  # CellComplex or Cluster
            # best-effort clarification using isinstance if available
            kind = "cellcomplex" if isinstance(topo, (TCellComplex,)) else ("cluster" if isinstance(topo, (TCluster,)) else "cellcomplex")
            tid = _new_id(kind)
            meta_entry = {"id": tid, "dict": topo_dict}
            meta["cellcomplexes" if kind == "cellcomplex" else "clusters"].append(meta_entry)
            node_name = "CellComplex" if kind == "cellcomplex" else "Cluster"
        elif faces and edges and verts:
            # could be a Cell
            kind = "cell" if isinstance(topo, (TCell,)) else "cell"
            tid = _new_id("cell")
            meta["cells"].append({"id": tid, "dict": topo_dict})
            node_name = "Cell"
        elif faces and verts:
            kind = "face"
            # This branch handles a single Face topology passed in
            # We will export its edges/verts below
            node_name = "FaceGroup"
        elif edges and verts and not faces:
            kind = "edge"
            node_name = "EdgeGroup"
        elif verts and not (edges or faces or cells):
            kind = "vertex"
            node_name = "VertexGroup"
        else:
            # unknown container, still export what we can find
            kind = "topology"
            node_name = name_hint

        # Build children according to rules
        child_node_ids = []

        # Cells (if any): each as its own subtree
        if cells:
            cell_children = []
            for i, c in enumerate(cells):
                c_node = export_topology(c, name_hint=f"Cell_{i}")
                if c_node is not None:
                    cell_children.append(c_node)
                # sidecar
                if isinstance(c, (TCell,)) is False:
                    # if Topology.Cells returned non-Cell, still register generically
                    meta["cells"].append({"id": _new_id("cell"), "dict": _entity_dictionary(c)})
            if cell_children:
                node = Node(name="Cells", children=cell_children)
                nodes.append(node)
                child_node_ids.append(len(nodes) - 1)

        # Faces (collection) if this node has many faces directly
        if faces and not cells:
            f_node = export_faces(topo, faces, topo)
            if f_node is not None:
                child_node_ids.append(f_node)

        # Edges
        if edges and not cells:
            e_node = export_edges(topo, edges, topo)
            if e_node is not None:
                child_node_ids.append(e_node)

        # Vertices
        if verts and not cells:
            v_node = export_vertices(topo, verts, topo)
            if v_node is not None:
                child_node_ids.append(v_node)

        # If this object looked like a single Face, also export its edges and vertices specifically
        if kind == "face" and isinstance(topo, (TFace,)):
            # export the single face geometry as its own primitive node
            n_face = export_face(topo, topo)
            if n_face is not None:
                child_node_ids.append(n_face)
            # export its edges and vertices
            try:
                fes = Topology.Edges(topo)
            except Exception:
                fes = []
            if fes:
                n_edges = export_edges(topo, fes, topo)
                if n_edges is not None:
                    child_node_ids.append(n_edges)
            try:
                fvs = Topology.Vertices(topo)
            except Exception:
                fvs = []
            if fvs:
                n_verts = export_vertices(topo, fvs, topo)
                if n_verts is not None:
                    child_node_ids.append(n_verts)

        # Create the node for this topology level
        node = Node(name=node_name, children=child_node_ids or None, extras={"topologyDictionary": topo_dict})
        nodes.append(node)
        return len(nodes) - 1

    # -------------------- drive export for all inputs --------------------
    root_children = []
    for i, topo in enumerate(topologies):
        idx = export_topology(topo, name_hint=f"Input_{i}")
        if idx is not None:
            root_children.append(idx)

    if not root_children:
        raise ValueError("No geometry or hierarchy to export.")

    # Scene
    root = Node(name="Root", children=root_children)
    nodes.append(root)
    gltf.nodes = nodes
    gltf.meshes = meshes
    gltf.scenes = [Scene(nodes=[len(nodes) - 1])]
    gltf.scene = 0

    # Buffers
    gltf.buffers = [Buffer(byteLength=len(binary_blob))]
    gltf.bufferViews = bufferViews
    gltf.accessors = accessors

    # -------------------- embed sidecar JSON metadata --------------------
    # Write meta JSON into a bufferView and point to it from asset.extras
    meta_json = json.dumps(meta, separators=(",", ":"), ensure_ascii=False).encode("utf-8")
    meta_start = _append_buffer(meta_json)
    bufferViews.append(BufferView(buffer=0, byteOffset=meta_start, byteLength=len(meta_json)))
    meta_bv_index = len(bufferViews) - 1
    gltf.asset.extras = gltf.asset.extras or {}
    gltf.asset.extras["topologicMetadata"] = {"bufferView": meta_bv_index, "mimeType": "application/json"}

    # Save
    os.makedirs(os.path.dirname(os.path.abspath(filepath)) or ".", exist_ok=True)
    gltf.set_binary_blob(bytes(binary_blob))
    gltf.save_binary(filepath)

    if not silent:
        print(f"Wrote {filepath}")
        print(f"  Points: {total_points}, Lines: {total_lines}, Triangles: {total_tris}")


In [27]:
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Color import Color
#cc = Cell.Prism(height=3, width=1.5)
cc = Cell.Torus()
faces = Topology.Faces(cc)
n = len(faces)
for i in range(n):
    r,g,b = Color.ByValueInRange(i, minValue=0, maxValue=n-1, colorScale="tropic")
    color = (r,g,b)
    d = Dictionary.ByKeysValues(["id", "color", "opacity"], [i, color, float(i)/float(n-1)])
    faces[i] = Topology.SetDictionary(faces[i], d)
cc = Topology.Triangulate(cc, transferDictionaries=True)
_ = ExportTopologicToGLB(cc, filepath=r"C:\Users\sarwj\Downloads\test.glb", faceColorKey="color", faceOpacityKey="opacity", include_edges=False, include_vertices=False)

Wrote C:\Users\sarwj\Downloads\test.glb
  Points: 0, Lines: 0, Triangles: 256
